# 🎓 Lecture 9: Knowledge Distillation - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/09_knowledge_distillation/demo.ipynb)

## What You'll Learn
- Teacher-student framework
- Soft labels and temperature scaling
- Different distillation methods
- Feature-based vs output-based distillation

In [ ]:
!pip install torch matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
print('Ready for Knowledge Distillation!')

## Part 1: The Idea of Knowledge Distillation

**Goal**: Transfer knowledge from a large "teacher" to a small "student".

**Key Insight**: Soft labels (probabilities) contain more information than hard labels.

Example: For image of "3", teacher outputs:
- "3": 0.7, "8": 0.2, "5": 0.05, ... (soft label - rich information!)
- vs hard label: just "3" (less information)

In [ ]:
# Visualize soft vs hard labels
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Hard label (one-hot)
hard_label = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
axes[0].bar(classes, hard_label, color='#ef4444')
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Probability')
axes[0].set_title('Hard Label (Ground Truth)\n"This is definitely a 3"', fontsize=12)
axes[0].set_ylim(0, 1.1)

# Soft label (teacher output)
soft_label = [0.01, 0.01, 0.02, 0.70, 0.01, 0.05, 0.01, 0.02, 0.15, 0.02]
axes[1].bar(classes, soft_label, color='#22c55e')
axes[1].set_xlabel('Class')
axes[1].set_ylabel('Probability')
axes[1].set_title('Soft Label (Teacher Output)\n"Mostly 3, but similar to 8 and 5"', fontsize=12)
axes[1].set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

print('💡 Soft labels tell the student:')
print('   - 3 looks similar to 8 (both have curves)')
print('   - 3 looks somewhat like 5 (upper curve)')
print('   - This is DARK KNOWLEDGE that hard labels miss!')

## Part 2: Temperature Scaling

Temperature softens the probability distribution:

$$p_i = \frac{\exp(z_i / T)}{\sum_j \exp(z_j / T)}$$

- T=1: Normal softmax
- T>1: Softer distribution (more knowledge transfer)
- T<1: Harder distribution

In [ ]:
def softmax_with_temperature(logits, temperature):
    """Apply softmax with temperature scaling."""
    return F.softmax(logits / temperature, dim=-1)

# Example logits
logits = torch.tensor([5.0, 2.0, 1.0, 0.5, 0.1, -0.5, -1.0, -1.5, 1.5, -2.0])

temperatures = [0.5, 1.0, 2.0, 5.0, 10.0]

fig, axes = plt.subplots(1, len(temperatures), figsize=(20, 4))

for ax, T in zip(axes, temperatures):
    probs = softmax_with_temperature(logits, T).numpy()
    ax.bar(classes, probs, color=plt.cm.coolwarm(T/10))
    ax.set_xlabel('Class')
    ax.set_ylabel('Probability')
    ax.set_title(f'T = {T}', fontsize=12)
    ax.set_ylim(0, 1)
    
    # Show entropy
    entropy = -np.sum(probs * np.log(probs + 1e-8))
    ax.text(0.5, 0.95, f'Entropy: {entropy:.2f}', transform=ax.transAxes, 
            ha='center', fontsize=10)

plt.suptitle('📊 Effect of Temperature on Softmax', fontsize=14)
plt.tight_layout()
plt.show()

print('\n💡 Higher temperature:')
print('   - More uniform distribution')
print('   - More information about class similarities')
print('   - Better for knowledge transfer')

## Part 3: Knowledge Distillation Loss

In [ ]:
def distillation_loss(student_logits, teacher_logits, labels, T=4.0, alpha=0.7):
    """
    Knowledge Distillation Loss:
    
    L = α * KL(soft_teacher || soft_student) * T² + (1-α) * CE(student, labels)
    
    Args:
        student_logits: Raw outputs from student
        teacher_logits: Raw outputs from teacher
        labels: Ground truth labels
        T: Temperature for soft labels
        alpha: Weight for distillation loss (vs CE loss)
    """
    # Soft targets from teacher
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    soft_student = F.log_softmax(student_logits / T, dim=1)
    
    # KL divergence (distillation loss)
    distill_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (T ** 2)
    
    # Cross-entropy with hard labels
    ce_loss = F.cross_entropy(student_logits, labels)
    
    # Combined loss
    total_loss = alpha * distill_loss + (1 - alpha) * ce_loss
    
    return total_loss, distill_loss, ce_loss

# Demo
batch_size = 32
num_classes = 10

teacher_logits = torch.randn(batch_size, num_classes) * 3  # Teacher outputs
student_logits = torch.randn(batch_size, num_classes)      # Student outputs
labels = torch.randint(0, num_classes, (batch_size,))

total, distill, ce = distillation_loss(student_logits, teacher_logits, labels)

print('📊 DISTILLATION LOSS COMPONENTS')
print('=' * 50)
print(f'Distillation Loss (KL): {distill.item():.4f}')
print(f'Cross-Entropy Loss: {ce.item():.4f}')
print(f'Total Loss (α=0.7): {total.item():.4f}')
print(f'\nFormula: L = 0.7 × KL + 0.3 × CE')

## Part 4: Complete Distillation Training

In [ ]:
# Define Teacher (large) and Student (small) models
class TeacherNet(nn.Module):
    """Large teacher model."""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(784, 1200), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(1200, 1200), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(1200, 800), nn.ReLU(), nn.Dropout(0.3),
        )
        self.classifier = nn.Linear(800, 10)
    
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

class StudentNet(nn.Module):
    """Small student model."""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(784, 400), nn.ReLU(),
            nn.Linear(400, 100), nn.ReLU(),
        )
        self.classifier = nn.Linear(100, 10)
    
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# Count parameters
teacher = TeacherNet()
student = StudentNet()

teacher_params = sum(p.numel() for p in teacher.parameters())
student_params = sum(p.numel() for p in student.parameters())

print('📊 MODEL SIZES')
print('=' * 40)
print(f'Teacher: {teacher_params:,} parameters')
print(f'Student: {student_params:,} parameters')
print(f'Compression: {teacher_params/student_params:.1f}x smaller')

In [ ]:
# Create synthetic data
X_train = torch.randn(2000, 784)
y_train = torch.randint(0, 10, (2000,))
X_test = torch.randn(500, 784)
y_test = torch.randint(0, 10, (500,))

def evaluate(model, X, y):
    model.eval()
    with torch.no_grad():
        return (model(X).argmax(1) == y).float().mean().item() * 100

# Train Teacher
print('📚 TRAINING TEACHER')
print('=' * 50)

teacher = TeacherNet()
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(30):
    teacher.train()
    optimizer.zero_grad()
    loss = criterion(teacher(X_train), y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        acc = evaluate(teacher, X_test, y_test)
        print(f'Epoch {epoch+1}: Loss={loss.item():.3f}, Acc={acc:.1f}%')

teacher_acc = evaluate(teacher, X_test, y_test)
print(f'\nFinal Teacher Accuracy: {teacher_acc:.1f}%')

In [ ]:
# Train Student with different methods
def train_student(student, method='scratch', teacher=None, epochs=30, T=4.0, alpha=0.7):
    """
    Train student with different methods:
    - scratch: Normal training with CE loss
    - distill: Knowledge distillation from teacher
    """
    optimizer = torch.optim.Adam(student.parameters(), lr=0.001)
    
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        student.train()
        optimizer.zero_grad()
        
        student_logits = student(X_train)
        
        if method == 'scratch':
            loss = F.cross_entropy(student_logits, y_train)
        else:  # distillation
            teacher.eval()
            with torch.no_grad():
                teacher_logits = teacher(X_train)
            loss, _, _ = distillation_loss(student_logits, teacher_logits, y_train, T, alpha)
        
        loss.backward()
        optimizer.step()
        
        acc = evaluate(student, X_test, y_test)
        history['loss'].append(loss.item())
        history['acc'].append(acc)
    
    return history

print('📚 TRAINING STUDENTS')
print('=' * 50)

# Student trained from scratch
print('\n1️⃣ Student trained from scratch...')
student_scratch = StudentNet()
history_scratch = train_student(student_scratch, method='scratch', epochs=30)

# Student with distillation
print('\n2️⃣ Student with knowledge distillation...')
student_distill = StudentNet()
history_distill = train_student(student_distill, method='distill', teacher=teacher, epochs=30)

print(f'\n📊 FINAL RESULTS')
print('=' * 50)
print(f'Teacher:              {teacher_acc:.1f}% ({teacher_params:,} params)')
print(f'Student (scratch):    {history_scratch["acc"][-1]:.1f}% ({student_params:,} params)')
print(f'Student (distilled):  {history_distill["acc"][-1]:.1f}% ({student_params:,} params)')

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy curves
axes[0].plot(history_scratch['acc'], label='From Scratch', color='#ef4444', linewidth=2)
axes[0].plot(history_distill['acc'], label='Distilled', color='#22c55e', linewidth=2)
axes[0].axhline(y=teacher_acc, color='#3b82f6', linestyle='--', label=f'Teacher ({teacher_acc:.1f}%)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Student Training Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Final comparison
methods = ['Teacher', 'Student\n(scratch)', 'Student\n(distilled)']
accuracies = [teacher_acc, history_scratch['acc'][-1], history_distill['acc'][-1]]
params = [teacher_params/1000, student_params/1000, student_params/1000]
colors = ['#3b82f6', '#ef4444', '#22c55e']

ax2 = axes[1]
bars = ax2.bar(methods, accuracies, color=colors)
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Final Accuracy Comparison')

for bar, p, acc in zip(bars, params, accuracies):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.1f}%\n({p:.0f}K params)', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print('\n💡 Distillation helps small model learn better!')

## Part 5: Temperature and Alpha Sensitivity

In [ ]:
# Test different hyperparameters
print('📊 HYPERPARAMETER SENSITIVITY')
print('=' * 50)

# Temperature sensitivity
print('\nTemperature sensitivity (α=0.7):')
temp_results = []
for T in [1, 2, 4, 8, 16]:
    student = StudentNet()
    history = train_student(student, method='distill', teacher=teacher, T=T, alpha=0.7, epochs=20)
    temp_results.append((T, history['acc'][-1]))
    print(f'  T={T:2d}: {history["acc"][-1]:.1f}%')

# Alpha sensitivity
print('\nAlpha sensitivity (T=4):')
alpha_results = []
for alpha in [0.0, 0.3, 0.5, 0.7, 0.9, 1.0]:
    student = StudentNet()
    history = train_student(student, method='distill', teacher=teacher, T=4, alpha=alpha, epochs=20)
    alpha_results.append((alpha, history['acc'][-1]))
    print(f'  α={alpha:.1f}: {history["acc"][-1]:.1f}%')

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

temps, temp_accs = zip(*temp_results)
axes[0].plot(temps, temp_accs, 'o-', color='#3b82f6', linewidth=2, markersize=8)
axes[0].set_xlabel('Temperature (T)')
axes[0].set_ylabel('Accuracy (%)')
axes[0].set_title('Temperature Sensitivity')
axes[0].grid(True, alpha=0.3)

alphas, alpha_accs = zip(*alpha_results)
axes[1].plot(alphas, alpha_accs, 'o-', color='#22c55e', linewidth=2, markersize=8)
axes[1].set_xlabel('Alpha (α)')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Alpha Sensitivity (KD vs CE weight)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('\n💡 Typical good values: T=4-8, α=0.5-0.9')

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. Soft labels contain "dark knowledge" about class similarities')
print('\n2. Temperature (T) controls how soft the labels are')
print('\n3. KD Loss = α × KL_div + (1-α) × CE_loss')
print('\n4. Distilled students often outperform scratch-trained')
print('\n5. Student can be 10x smaller with similar accuracy')
print('\n6. T=4-8 and α=0.5-0.9 usually work well')
print('\n' + '=' * 60)
print('\n📚 Next: MCUNet and TinyML!')